# Bibliotecas

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import shapely as sh
from my_example import *

# Função Objetivo

In [ ]:
# Leitura do dataframe

# Admissible stress of the soil
n_comb = 3
df_base = pd.read_excel('teste_reduzido.xlsx')
df = tensao_adm_solo(df_base)
n_fundacoes = len(df_base)
args = (df_base, n_comb)

def obj(x, args):
    df_base = args[0]
    n_comb = args[1]  
    df = df_base.copy(deep=True)


    # Constantes
    ro = 0.01
    cob = 0.025
    fck = 25000
    fcd = fck / 1.4

    # Variáveis de projeto
    # h_x = x[0:n_fundacoes]
    # h_y = x[n_fundacoes:2*n_fundacoes]
    # h_z = [x[2*n_fundacoes:3*n_fundacoes]]
    h_x = x.copy()
    h_y = x.copy()
    h_z = [0.6] * n_fundacoes
    # h_x_aux = [0.6, 0.7, 0.8, 0.9]
    # h_y_aux = [0.6, 0.7, 0.8, 0.9]
    # h_z_aux = [0.6, 0.7, 0.8, 0.9]

    # h_x = h_x_aux[round(h_x)]
    # h_y = h_y_aux[round(h_y)]
    # h_z = h_z_aux[round(h_z)]

    # Para calculo da tensão max e minima
    max_tensoes = []
    min_tensoes = []

    # Lista para armazenar os esforços da combinação crítica
    index_critico_max = []
    index_critico_min = []
    fz_critico_max = []; mx_critico_max = []; my_critico_max = []
    fz_critico_min = []; mx_critico_min = []; my_critico_min = []

    # Atribuindo as dimensões da fundação
    df['h_x (m)'] = h_x
    df['h_y (m)'] = h_y
    df['h_z (m)'] = h_z

    # Volume da fundação
    df['volume (m3)'] = df['h_x (m)'] * df['h_y (m)'] * df['h_z (m)']

    # Lista para armazenar os rótulos de combinação
    rotulos_comb = [f'c{i}' for i in range(1, n_comb + 1)]

    # Verificando a tensão em todos os itens de fundação
    for idx, row in df.iterrows(): 
        t_max_aux = []
        t_min_aux = []

        # # Check stress max and min for all load combinations
        for i in range(1, n_comb + 1):
            aux = f'c{i}'
            t_max_val, t_min_val = calcular_sigma_max(
                                                        row[f'Fz-{aux}'],
                                                        row[f'Mx-{aux}'],
                                                        row[f'My-{aux}'],
                                                        row['h_x (m)'],
                                                        row['h_y (m)'],
                                                    )
            df[['tensao max. (kPa)', 'tensao min. (kPa)']] = df.apply(lambda row: calcular_sigma_max_min(row[f'Fz-{aux}'], row[f'Mx-{aux}'], row[f'My-{aux}'], row['h_x (m)'], row['h_y (m)']), axis=1, result_type='expand')
            t_max_aux.append(t_max_val)
            t_min_aux.append(t_min_val)
    
            
        # Armazenando os valores máximos e mínimos
        max_tensoes.append(max(t_max_aux))
        min_tensoes.append(min(t_min_aux))

        # Armazenando os índices das combinações críticas
        idx_max = t_max_aux.index(max(t_max_aux))
        idx_min = t_min_aux.index(min(t_min_aux))
        comb_max = rotulos_comb[idx_max]
        comb_min = rotulos_comb[idx_min]


        index_critico_max.append(comb_max)
        index_critico_min.append(comb_min)

        # Armazenando os esforços das combinações críticas
        fz_critico_max.append(row[f'Fz-{comb_max}']); mx_critico_max.append(row[f'Mx-{comb_max}']); my_critico_max.append(row[f'My-{comb_max}'])
        fz_critico_min.append(row[f'Fz-{comb_min}']); mx_critico_min.append(row[f'Mx-{comb_min}']); my_critico_min.append(row[f'My-{comb_min}'])

    # Adiciona as colunas ao df
    df['min_tensao (kPa)'] = min_tensoes
    df['max_tensao (kPa)'] = max_tensoes

    df['comb_max_tensao'] = index_critico_max
    df['Fz_max (kN)'] = fz_critico_max
    df['Mx_max (kN.m)'] = mx_critico_max
    df['My_max (kN.m)'] = my_critico_max

    df['comb_min_tensao'] = index_critico_min
    df['Fz_min (kN)'] = fz_critico_min
    df['Mx_min (kN.m)'] = mx_critico_min
    df['My_min (kN.m)'] = my_critico_min

    ## Para restrição geométrica de balanço da sapata
    # lista para armazenar resultados
    g0_geo_pilar_sapata = []
    g1_geo_pilar_sapata = []
    g2_geo_pilar_sapata = []
    g3_geo_pilar_sapata = []
    cap = [] #valor do balanço em x
    cbp = [] #valor do balanço em y

    for idx, row in df.iterrows(): 
        cap_value, cbp_value, g0, g1, g2, g3 = restricao_geometrica_balanco_pilar_sapata(
                                                                                            h_x=row['h_x (m)'],
                                                                                            h_y=row['h_y (m)'],
                                                                                            h_z=row['h_z (m)'],
                                                                                            a_p=row['ap (m)'],
                                                                                            b_p=row['bp (m)']
                                                                                        )
        # Armazena cada valor em listas
        cap.append(cap_value)
        cbp.append(cbp_value)
        g0_geo_pilar_sapata.append(g0)
        g1_geo_pilar_sapata.append(g1)
        g2_geo_pilar_sapata.append(g2)
        g3_geo_pilar_sapata.append(g3)

    # Adiciona comprimento do balanço ao DataFrame
    df['cap (m)'] = cap
    df['cbp (m)'] = cbp

    ##Para restrição geometrica Pilar_Sapata
    g0_geo_pilar = []
    g1_geo_pilar = []

    for idx, row in df.iterrows(): 
        g0, g1 = restricao_geometrica_pilar_sapata(
                                                    h_x=row['h_x (m)'],
                                                    h_y=row['h_y (m)'],
                                                    a_p=row['ap (m)'],
                                                    b_p=row['bp (m)']
                                                )
        # Armazena cada valor em listas
        g0_geo_pilar.append(g0)
        g1_geo_pilar.append(g1)

    ## para restrição de punção
    # Atribuindo os parâmetros
    df['ro'] = ro
    df['cob (m)'] = cob
    df['fck (kPa)'] = fck
    df['fcd (kPa)'] = fcd

    # lista para armazenar resultados
    ke_puncao = []
    kx_puncao = []
    ky_puncao = []
    wpx_puncao = []
    wpy_puncao = []
    u_puncao = []
    talsd1_puncao = []
    talrd1_puncao = []
    talsd2_puncao = []
    talrd2_puncao = []
    g0_puncao = []
    g1_puncao = []
    g2_puncao = []
    g3_puncao = []
    g4_puncao = []
    g5_puncao = []
    g6_puncao = []

    for idx, row in df.iterrows(): 
        ke, kx, ky, wpx, wpy, u, talsd1, talrd1, talsd2, talrd2, g0, g1, g2, g3, g4, g5, g6 = restricao_puncao(
                                                                                                                h_x=row['h_x (m)'],
                                                                                                                h_y=row['h_y (m)'],
                                                                                                                h_z=row['h_z (m)'],
                                                                                                                a_p=row['ap (m)'],
                                                                                                                b_p=row['bp (m)'],
                                                                                                                f_z=row['Fz_max (kN)'], #verificar se é max ou min
                                                                                                                m_x=row['Mx_max (kN.m)'], #verificar se é max ou min
                                                                                                                m_y=row['My_max (kN.m)'], #verificar se é max ou min
                                                                                                                ro=row['ro'],
                                                                                                                cob=row['cob (m)'],
                                                                                                                fck=row['fck (kPa)'],
                                                                                                                fcd=row['fcd (kPa)']
                                                                                                            )
        # Armazena cada valor em listas
        ke_puncao.append(ke)
        kx_puncao.append(kx)
        ky_puncao.append(ky)
        wpx_puncao.append(wpx)
        wpy_puncao.append(wpy)
        u_puncao.append(u)  
        talsd1_puncao.append(talsd1)
        talrd1_puncao.append(talrd1)
        talsd2_puncao.append(talsd2)
        talrd2_puncao.append(talrd2)
        g0_puncao.append(g0)
        g1_puncao.append(g1)
        g2_puncao.append(g2)
        g3_puncao.append(g3)
        g4_puncao.append(g4)
        g5_puncao.append(g5)
        g6_puncao.append(g6)

    df['kx_puncao'] = kx_puncao
    df['ky_puncao'] = ky_puncao
    df['ke_puncao'] = ke_puncao
    df['wpx_puncao'] = wpx_puncao
    df['wpy_puncao'] = wpy_puncao
    df['u_puncao'] = u_puncao
    df['talsd1_puncao'] = talsd1_puncao
    df['talrd1_puncao'] = talrd1_puncao
    df['talsd2_puncao'] = talsd2_puncao
    df['talrd2_puncao'] = talrd2_puncao

    # Calculando os vertices A, B, C e D da sapata no df    
    df['x_a'] = df['xg (m)'] + df['h_x (m)'] / 2
    df['y_a'] = df['yg (m)'] + df['h_y (m)'] / 2
    df['x_b'] = df['xg (m)'] - df['h_x (m)'] / 2
    df['y_b'] = df['yg (m)'] - df['h_y (m)'] / 2
    df['x_c'] = df['xg (m)'] - df['h_x (m)'] / 2
    df['y_c'] = df['yg (m)'] + df['h_y (m)'] / 2
    df['x_d'] = df['xg (m)'] + df['h_x (m)'] / 2
    df['y_d'] = df['yg (m)'] - df['h_y (m)'] / 2
    coords_list = [f"{row['x_a']},{row['y_a']},{row['x_b']},{row['y_b']},{row['x_c']},{row['y_c']},{row['x_d']},{row['y_d']}" for _, row in df.iterrows()]
    df['coords'] = coords_list
    poligonos = []
    for coord_string in df['coords']:
        # Divide a string e converte para números
        numeros = [float(x) for x in coord_string.split(',')]
        # Cria os pontos do polígono: A -> B -> C -> D -> A (fechando)
        pontos = [
            (numeros[0], numeros[1]),  # Ponto A
            (numeros[2], numeros[3]),  # Ponto B  
            (numeros[4], numeros[5]),  # Ponto C
            (numeros[6], numeros[7]),  # Ponto D
            (numeros[0], numeros[1])   # Volta para A (fechar o polígono)
        ]
        poligono = sh.geometry.Polygon(pontos)
        poligonos.append(poligono)
    n = len(poligonos)
    contagem_intersecoes = [0] * n  # Lista para armazenar as contagens

    # Para cada polígono, verifica com todos os outros
    for i in range(n):
        for j in range(n):
            if i != j:  # Não comparar com ele mesmo
                if poligonos[i].intersects(poligonos[j]):
                    contagem_intersecoes[i] += 1

    # Colocando as restrições no df

    # Checagem de tensões 
    # Checagem tensão mínima
    df['g_0'] = np.where(
                        df['min_tensao (kPa)'] < 0,
                        np.abs(df['min_tensao (kPa)'] / df['sigma_adm (kPa)']),        # Comportamento se valores negativos
                        1.15 * df['min_tensao (kPa)'] / df['sigma_adm (kPa)'] - 1       # Comportamento se valores >= 0
                        )

    # Checagem tensão máxima
    df['g_1'] = np.where(
                        df['max_tensao (kPa)'] < 0,
                        np.abs(df['max_tensao (kPa)'] / df['sigma_adm (kPa)']),        # Comportamento se valores negativos
                        1.15 * df['max_tensao (kPa)'] / df['sigma_adm (kPa)'] - 1       # Comportamento se valores >= 0
                        )

    # Restrições geométricas de balanço
    df['g_2'] = g0_geo_pilar_sapata   # limite superior (hx)
    df['g_3'] = g1_geo_pilar_sapata   # limite superior (hy)
    df['g_4'] = g2_geo_pilar_sapata  # limite inferior (hx)
    df['g_5'] = g3_geo_pilar_sapata   # limite inferior (hy)

    # Restrições geométricas de pilar_sapata
    df['g_6'] = g0_geo_pilar
    df['g_7'] = g1_geo_pilar

    # Restrições geométricas de punção
    df['g_8'] = g0_puncao
    df['g_9'] = g1_puncao
    df['g_10'] = g2_puncao
    df['g_11'] = g3_puncao
    df['g_12'] = g4_puncao
    df['g_13'] = g5_puncao
    df['g_14'] = g6_puncao

    # Restrições de interseção
    df['g_15'] = contagem_intersecoes

    # Lista para armazenar as penalizações
    g = []

    # Aplicando penalizações às restrições
    for idx, row in df.iterrows(): 
        g_0 = row['g_0'] * 1e6 if row['g_0'] > 0 else 0
        g_1 = row['g_1'] * 1e6 if row['g_1'] > 0 else 0
        g_2 = row['g_2'] * 1e6 if row['g_2'] > 0 else 0
        g_3 = row['g_3'] * 1e6 if row['g_3'] > 0 else 0
        g_4 = row['g_4'] * 1e6 if row['g_4'] > 0 else 0
        g_5 = row['g_5'] * 1e6 if row['g_5'] > 0 else 0
        g_6 = row['g_6'] * 1e6 if row['g_6'] > 0 else 0
        g_7 = row['g_7'] * 1e6 if row['g_7'] > 0 else 0
        g_8 = row['g_8'] * 1e6 if row['g_8'] > 0 else 0
        g_9 = row['g_9'] * 1e6 if row['g_9'] > 0 else 0
        g_10 = row['g_10'] * 1e6 if row['g_10'] > 0 else 0
        g_11 = row['g_11'] * 1e6 if row['g_11'] > 0 else 0
        g_12 = row['g_12'] * 1e6 if row['g_12'] > 0 else 0
        g_13 = row['g_13'] * 1e6 if row['g_13'] > 0 else 0
        g_14 = row['g_14'] * 1e6 if row['g_14'] > 0 else 0
        g_15 = row['g_15'] * 1e6 if row['g_15'] > 0 else 0  
        
        # Somando todas as penalizações e adicionando à lista
        g.append(g_0 + g_1 + g_2 + g_3 + g_4 + g_5 + g_6 + g_7 + g_8 + g_9 + g_10 + g_11 + g_12 + g_13 + g_14 + g_15)

    # Adicionando a coluna de penalização ao df
    df['g'] = g

    # Calculando o volume total
    vol = df['volume (m3)'].sum() 
    
    return vol, df['g'].values

In [9]:
h_x = np.full(n_fundacoes, 2.0)  # todas 3.0 m
h_y = np.full(n_fundacoes, 2.5)  # todas 3.0 m
h_z = np.full(n_fundacoes, 1.0)  # todas 1.0 m

x = np.concatenate((h_x, h_y, h_z))
# h_x =  2
# h_y =  2.5
# h_z =  1
# Empilha em sequência
# x = np.hstack([2, 2, 2, 2,2 ,2.5, 2.5, 2.5, 2.5, 2.5, 1, 1, 1, 1, 1])
# x = [2, 2, 2, 2,2 ,2.5, 2.5, 2.5, 2.5, 2.5, 1, 1, 1, 1, 1]

f, g = obj(x)
print(f)
print(g)


TypeError: obj() missing 1 required positional argument: 'args'

In [8]:
from pymoo.core.problem import ElementwiseProblem

from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling

from pymoo.optimize import minimize

# Leitura do dataframe
n_comb = 3
df = pd.read_excel('teste_reduzido.xlsx')
num_linhas = len(df)
n_fundacoes = num_linhas
df = tensao_adm_solo(df)

# 1. Definindo o Problema
class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=n_fundacoes * 3,
                         n_obj=1,
                         n_constr=num_linhas,
                         xl=np.array([0.6] * n_fundacoes * 3),
                         xu=np.array([3.0] * n_fundacoes * 3))

    def _evaluate(self, x, out):
        f, g = obj(x)

        out["F"] = f
        out["G"] = g

problem = MyProblem()

# 2. Configurando o Algoritmo
algorithm = GA(pop_size=40,
               n_offsprings=10,
               sampling=FloatRandomSampling(),
               crossover=SBX(prob=0.9, eta=15),
               mutation=PM(eta=20),
               eliminate_duplicates=True
)

#3. Executando a Otimização
res = minimize(problem,
               algorithm,
               ('n_gen', 40),
               seed=1,
               verbose=False)

X = res.X
F = res.F
G = res.G

print(X)
print(F)
print(G)

None
None
None


# Tensão no solo

In [27]:
n_comb = 3
df = pd.read_excel('teste_wand.xlsx')
df = tensao_adm_solo(df)
df

,Elemento,ap (m),bp (m),spt,solo,xg (m),yg (m),Fz-c1,Mx-c1,My-c1,Fz-c2,Mx-c2,My-c2,Fz-c3,Mx-c3,My-c3,sigma_adm (kPa)
0,P01,0.25,1.20,10,argila,2.900,25.255,485.9,-0.3,4.1,511.6,-32.4,-0.4,511.6,-32.4,-0.4,200.0
1,P02,0.30,1.50,12,argila,7.985,25.105,885.8,-1.0,9.2,912.1,-65.4,0.1,915.9,-40.0,0.1,240.0
2,P03,0.35,2.32,30,argila,13.875,25.515,1314.0,-3.9,20.4,1696.4,-214.1,0.8,1698.9,-214.6,0.7,600.0
3,P04,0.30,1.50,35,argila,24.850,25.105,855.5,-3.7,9.2,891.9,-60.0,0.6,908.5,-36.9,0.4,700.0
4,P05,0.25,1.20,45,argila,24.850,25.255,478.6,-3.1,5.0,496.0,-27.6,0.7,508.0,-27.6,0.7,900.0
5,P06,0.25,1.75,45,argila,0.000,24.980,259.9,0.7,5.9,373.8,-85.5,-0.1,390.8,-85.1,-0.1,900.0
6,P07,0.25,1.75,50,argila,27.750,24.980,380.8,-6.1,6.2,402.4,-70.0,0.5,402.4,-70.0,0.5,1000.0
7,P08,2.10,0.25,12,argila,8.125,19.590,857.4,-0.2,115.6,913.0,-0.9,0.1,950.6,-0.9,-0.5,240.0
8,P09,2.10,0.25,32,argila,19.765,19.590,863.4,-0.2,113.8,863.4,-0.2,113.8,903.4,-0.2,112.5,640.0
9,P10,0.25,1.20,90,argila,0.000,18.065,230.0,1.4,4.4,234.0,-35.4,0.0,366.3,-0.8,-4.2,1800.0


In [5]:
df

,Elemento,ap (m),bp (m),spt,solo,xg (m),yg (m),Fz-c1,Mx-c1,My-c1,Fz-c2,Mx-c2,My-c2,Fz-c3,Mx-c3,My-c3,sigma_adm (kPa)
0,P01,0.25,0.15,10,argila,1.0,1.0,1000,30,2,200,-30,-1,500,-2,5,200.0
1,P02,0.30,0.15,10,argila,2.0,2.0,1000,30,2,200,-30,-1,500,-2,5,200.0
2,P03,0.30,0.15,10,argila,2.5,2.5,1000,30,2,200,-30,-1,500,-2,5,200.0
